In [1]:
import pandas as pd
import commonlib.prepdata as prepdata
import commonlib.gencharts as gencharts
import matplotlib.pyplot as plt
import commonlib.config as cfg

In [ ]:
c = cfg.Config()
print(c.TERRAFORM_LOG_PATH)
parsed_records=prepdata.read_json_from_file(c.TERRAFORM_LOG_PATH)
normalized_records=prepdata.normalize_records(parsed_records)
df = pd.json_normalize(normalized_records)
unique_types = df['type'].drop_duplicates().tolist()

print(unique_types)
df_refresh_start = df[df['type'] == 'refresh_start']
df_refresh_start = df_refresh_start.rename(columns={'timestamp': 'refresh_start_timestamp'})

df_refresh_complete = df[df['type'] == 'refresh_complete']
df_refresh_complete = df_refresh_complete.rename(columns={'timestamp': 'refresh_complete_timestamp'})


df_merged_refresh = pd.merge(df_refresh_start[['resource_id', 
'refresh_start_timestamp','resource','resource_type','resource_name']],
                             df_refresh_complete[['resource_id', 'refresh_complete_timestamp']], 
                             on='resource_id')

df_merged_refresh['refresh_start_datetime'] = pd.to_datetime(df_merged_refresh['refresh_start_timestamp'])
df_merged_refresh['refresh_complete_datetime'] = pd.to_datetime(df_merged_refresh['refresh_complete_timestamp'])

# calculate time difference in minutes
df_merged_refresh['time_diff_minutes'] = (df_merged_refresh['refresh_complete_datetime'] - df_merged_refresh['refresh_start_datetime']).dt.total_seconds() / 60

df_apply_start = df[df['type'] == 'apply_start']
df_apply_start = df_refresh_complete.rename(columns={'timestamp': 'apply_start_timestamp'})

df_apply_complete = df[df['type'] == 'apply_complete']
df_apply_complete = df_refresh_complete.rename(columns={'timestamp': 'apply_complete_timestamp'})

df_apply_progress = df[df['type'] == 'apply_progress']
df_apply_resource_drift = df[df['type'] == 'resource_drift']
df_apply_planned_change = df[df['type'] == 'planned_change']

##
Type Analysis

In [ ]:
gencharts.generate_plt_by_resource_type(df,"refresh_start")
gencharts.generate_plt_by_resource_type(df,"refresh_complete")
gencharts.generate_plt_by_resource_type(df,"resource_drift")
gencharts.generate_plt_by_resource_type(df,"planned_change")

gencharts.generate_plt_by_resource_type(df,"apply_progress")

In [ ]:
df_merged_refresh[['resource', 'refresh_start_timestamp','refresh_complete_timestamp','time_diff_minutes']].copy().sort_values(by='time_diff_minutes', ascending=False).head(20)